In [109]:
### the trip ids dont match in our processed data and Ayyas data
### we need to match the trip ids

In [110]:
#mount google drive
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [111]:
dataOutPath = '/content/drive/MyDrive/DSE_Project/dataOut/'
AyyasProcessedDataPath = '/content/drive/MyDrive/DSE_Project/AyyasProcessedData/'

In [112]:
import numpy as np
import pandas as pd

Mydata = pd.read_csv(dataOutPath + 'bus_trip_all_points.csv')

AyyasData =pd.read_csv(AyyasProcessedDataPath + 'bus_running_times_feature_added_all.csv')
Aayyas_dwelltime_df = pd.read_csv(AyyasProcessedDataPath +'bus_stop_times_feature_added_all.csv')

In [113]:
AyyasData.dropna(inplace=True)

In [114]:
# show all the columns in pandas dataframes
pd.set_option('display.max_columns', None)

In [ ]:
# AyyasDataonly keep 'trip_id', 'deviceid', 'direction', 'date', 'start_time', 'end_time' columns
AyyasData_forComp = AyyasData[['trip_id', 'deviceid',  'date', 'start_time', 'end_time']]

AyyasData_forComp['start_time'] = pd.to_datetime(AyyasData_forComp['date'] + ' ' + AyyasData_forComp['start_time'])
AyyasData_forComp['end_time'] = pd.to_datetime(AyyasData_forComp['date'] + ' ' + AyyasData_forComp['end_time'])
AyyasData_forComp['date'] = pd.to_datetime(AyyasData_forComp['date'], format='%Y-%m-%d').dt.date


In [ ]:
AyyasData_forComp.dropna(inplace=True)

In [117]:


# Group by 'trip_id' and aggregate
AyyasData_forComp = AyyasData_forComp.groupby(['trip_id']).agg(
    deviceid=pd.NamedAgg(column='deviceid', aggfunc='first'),
    date=pd.NamedAgg(column='date', aggfunc='first'),
    start_time=pd.NamedAgg(column='start_time', aggfunc='min'),
    end_time=pd.NamedAgg(column='end_time', aggfunc='max')
).reset_index()



In [ ]:
AyyasData_forComp

In [119]:
# get a list of all columns in the dataframe
colls=list(Mydata.columns)
print(colls)

['id', 'deviceid', 'devicetime', 'latitude', 'longitude', 'speed', 'date', 'time', 'geometry', 'bus_stop', 'trip_id', 'direction']


In [ ]:
Mydata_forComp=Mydata[['trip_id', 'deviceid', 'date', 'time']]
# Convert 'start_time' and 'end_time' columns to datetime format ( start tieme and end time only have time in hh:mm:ss format)
Mydata_forComp['time'] = pd.to_datetime(Mydata['date'] + ' ' + Mydata['time'])
# convert 'date' column to datetime format
Mydata_forComp['date'] = pd.to_datetime(Mydata['date'], format='%Y-%m-%d').dt.date


In [ ]:
# group Mydata by trip id and pick the middle value of every column , columns are ['trip_id', 'deviceid', 'date', 'time']

# Sort the data by 'trip_id' and 'time'
Mydata_forComp = Mydata_forComp.sort_values(by=['trip_id', 'time'])

# Calculate the number of rows in each 'trip_id' group
group_sizes = Mydata_forComp.groupby('trip_id').size().reset_index(name='group_size')

# Select the middle row from each 'trip_id' group
Mydata_forComp = Mydata_forComp.merge(group_sizes, on='trip_id')
Mydata_forComp['middle_row'] = Mydata_forComp['group_size'] // 2


In [ ]:
# Select the middle values for each 'trip_id' group
Mydata_forComp = Mydata_forComp[Mydata_forComp.groupby('trip_id').cumcount() == Mydata_forComp['middle_row']]
Mydata_forComp

In [ ]:


# Cleanup: Drop the helper columns
Mydata_forComp.drop(columns=['group_size', 'middle_row'], inplace=True)

# Reset the index
Mydata_forComp.reset_index(drop=True, inplace=True)


In [133]:
# make AyyasData_forComp deviceid integer without the decimal point
AyyasData_forComp['deviceid'] = AyyasData_forComp['deviceid'].astype(int)


In [ ]:
smaller_df, larger_df = AyyasData_forComp, Mydata_forComp

# Create a mapping dictionary based on common bus IDs, date, and time
mapping_dict = {}
for index, row in smaller_df.iterrows():
    matching_rows = larger_df[
        (larger_df['deviceid'] == row['deviceid']) &
        (larger_df['time'] >= row['start_time']) &
        (larger_df['time'] <= row['end_time'])
    ]
    if len(matching_rows) >= 1:
        mapping_dict[row['trip_id']] = matching_rows.iloc[0]['trip_id']

# Update trip IDs in the smaller data frame using the mapping dictionary
smaller_df['trip_id'] = smaller_df['trip_id'].map(mapping_dict)

# Now, the trip IDs in the smaller data frame have been updated to match the larger data frame
print(smaller_df)


In [97]:
# # ### this runs faster than above code

# smaller_df, larger_df = AyyasData_forComp, Mydata_forComp
# # Step 1: Merge DataFrames
# merged_df = pd.merge(
#     smaller_df[['trip_id', 'deviceid', 'start_time', 'end_time']],
#     larger_df[['trip_id', 'deviceid', 'time']],
#     on='deviceid',
#     how='left'
# )

# # Filter rows based on time conditions
# valid_rows = merged_df[
#     (merged_df['time'] >= merged_df['start_time']) &
#     (merged_df['time'] <= merged_df['end_time'])
# ]

# # Step 2: Create Mapping
# mapping_dict = pd.Series(
#     valid_rows['trip_id_y'].values,
#     index=valid_rows['trip_id_x']
# ).to_dict()

# #Step 3: Update Trip IDs
# smaller_df['trip_id'] = smaller_df['trip_id'].map(mapping_dict)


In [135]:
Aayyas_dwelltime_df['trip_id'] = Aayyas_dwelltime_df['trip_id'].map(mapping_dict)


In [ ]:
Aayyas_dwelltime_df

In [136]:
# Count the number of unique key-value pairs in mapping_dict
unique_pairs = len({(key, value) for key, value in mapping_dict.items()})

print("Number of unique key-value pairs in mapping_dict:", unique_pairs)


Number of unique key-value pairs in mapping_dict: 826


In [137]:
# drop null rows AyyasData
smaller_df.dropna(inplace=True)

In [138]:
AyyasData['trip_id'] = AyyasData['trip_id'].map(mapping_dict)


In [ ]:
# drop raws where only trip_id is null
AyyasData.dropna(subset=['trip_id'], inplace=True)

Aayyas_dwelltime_df.dropna(subset=['trip_id'], inplace=True)

In [ ]:
Aayyas_dwelltime_df

In [ ]:
AyyasData

In [ ]:
# print number of unique trip_id in Mydata
print(Mydata['trip_id'].nunique())

15891


In [ ]:
# get a list of values in mapping_dict dictionary
available_trip_ids=list(mapping_dict.values())

In [ ]:
available_trip_ids=list(set(available_trip_ids))

In [ ]:
# drop the rows in Mydata where trip_id is not in available_trip_ids
Mydata=Mydata[Mydata['trip_id'].isin(available_trip_ids)]

In [ ]:
Mydata

In [ ]:
# get my data trip id == 4890
AyyasData[AyyasData['trip_id']==2370]

In [ ]:
# save my data to csv file bus_trip_all_points_moddedDF.csv
Mydata.to_csv('../DataOut/bus_trip_all_points_moddedDF.csv', index=False)

# save AyyasData to csv file bus_running_times_feature_added_all_moddedDF.csv
AyyasData.to_csv('../DataOut/bus_running_times_feature_added_all_moddedDF.csv', index=False)

# save AyyasData to csv file bus_running_times_feature_added_all_moddedDF.csv
Aayyas_dwelltime_df.to_csv('../DataOut/bus_stop_times_feature_added_all_moddedDF.csv', index=False)

